# ETL y EDA de Dataset homicidios 

<br>

Importamos las librerías

In [43]:
import pandas as pd
import numpy as np
import seaborn as sns
from datetime import datetime
import geopandas as gpd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

<br>

#### Dentro del Dataset provisto en formato xlsx, existen 2 pestañas con datos. Se procede a la extracción de las pestañas para su análisis

<br>

Se extraen los datos de la pestaña HECHOS del excel provisto.

In [44]:
# Se extrae la pestaña HECHOS del dataset HOMICIDIOS
df_homicidios_hechos = pd.read_excel(r"C:\Users\Cebol\OneDrive\Escritorio\PI_02_siniestros\Datasets\homicidios.xlsx", sheet_name='HECHOS')

# Se imprimen en pantalla las primeras filas
df_homicidios_hechos.head()

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,...,Altura,Cruce,Dirección Normalizada,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO
0,2016-0001,1,2016-01-01,2016,1,1,04:00:00,4,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,...,NaN,"FERNANDEZ DE LA CRUZ, F., GRAL. AV.","PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...",8,Point (98896.78238426 93532.43437792),-58.47533969,-34.68757022,MOTO-AUTO,MOTO,AUTO
1,2016-0002,1,2016-01-02,2016,1,2,01:15:00,1,AV GRAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,...,NaN,DE LOS CORRALES AV.,"PAZ, GRAL. AV. y DE LOS CORRALES AV.",9,Point (95832.05571093 95505.41641999),-58.50877521,-34.66977709,AUTO-PASAJEROS,AUTO,PASAJEROS
2,2016-0003,1,2016-01-03,2016,1,3,07:00:00,7,AV ENTRE RIOS 2034,AVENIDA,...,2034.0,NaN,ENTRE RIOS AV. 2034,1,Point (106684.29090040 99706.57687843),-58.39040293,-34.63189362,MOTO-AUTO,MOTO,AUTO
3,2016-0004,1,2016-01-10,2016,1,10,00:00:00,0,AV LARRAZABAL Y GRAL VILLEGAS CONRADO,AVENIDA,...,NaN,"VILLEGAS, CONRADO, GRAL.","LARRAZABAL AV. y VILLEGAS, CONRADO, GRAL.",8,Point (99840.65224780 94269.16534422),-58.46503904,-34.68092974,MOTO-SD,MOTO,SD
4,2016-0005,1,2016-01-21,2016,1,21,05:20:00,5,AV SAN JUAN Y PRESIDENTE LUIS SAENZ PEÑA,AVENIDA,...,NaN,"SAENZ PE?A, LUIS, PRES.","SAN JUAN AV. y SAENZ PEÃ‘A, LUIS, PRES.",1,Point (106980.32827929 100752.16915795),-58.38718297,-34.62246630,MOTO-PASAJEROS,MOTO,PASAJEROS


<br>
<br>

Se observan los tipos de datos y cantidad de nulos

In [45]:
dicc = {"Campo": [], "Tipo_de_dato": [], "Nulos": []}

for columna in df_homicidios_hechos.columns:
    dicc["Campo"].append(columna)
    dicc["Tipo_de_dato"].append(df_homicidios_hechos[columna].apply(type).unique())
    dicc["Nulos"].append(df_homicidios_hechos[columna].isnull().sum())

df_info = pd.DataFrame(dicc)
df_info

,Campo,Tipo_de_dato,Nulos
0,ID,[<class 'str'>],0
1,N_VICTIMAS,[<class 'int'>],0
2,FECHA,[<class 'pandas._libs.tslibs.timestamps.Timest...,0
3,AAAA,[<class 'int'>],0
4,MM,[<class 'int'>],0
5,DD,[<class 'int'>],0
6,HORA,"[<class 'datetime.time'>, <class 'str'>, <clas...",0
7,HH,"[<class 'int'>, <class 'str'>]",0
8,LUGAR_DEL_HECHO,[<class 'str'>],0
9,TIPO_DE_CALLE,[<class 'str'>],0


<br>

En las columnas 'Altura' y 'Cruce' se encuentran la mayor cantidad de datos faltantes, debido a que se completa cuando el hecho ocurre en algún punto de la calle (Altura) o en la intersección de dos calles (Cruce). Se procede a completar los datos faltantes en la columna 'Dirección normalizada', luego eliminaré las columnas 'LUGAR_DEL_HECHO', 'Calle' y 'Altura'. Finalmente se cambiará el nombre de la columna 'Dirección Normalizada' por 'Lugar del Hecho'.

<br>


In [46]:
# Se rellenan los valores faltantes en la columna 'Dirección normalizada' con los valores de la columna 'LUGAR_DEL_HECHO' 
df_homicidios_hechos['Dirección Normalizada'].fillna(df_homicidios_hechos['LUGAR_DEL_HECHO'], inplace=True)

# Se chequea la cantidad de nulos en la columna 'Dirección Normalizada'
df_homicidios_hechos['Dirección Normalizada'].isna().sum()

0

<br>
<br>

Se procede a eliminar las columnas

In [47]:
# Se eliminan las columnas
df_homicidios_hechos = df_homicidios_hechos.drop(['LUGAR_DEL_HECHO', 'Calle', 'Altura'], axis=1)

# Se confirma 
df_homicidios_hechos.columns

Index(['ID', 'N_VICTIMAS', 'FECHA', 'AAAA', 'MM', 'DD', 'HORA', 'HH',
       'TIPO_DE_CALLE', 'Cruce', 'Dirección Normalizada', 'COMUNA',
       'XY (CABA)', 'pos x', 'pos y', 'PARTICIPANTES', 'VICTIMA', 'ACUSADO'],
      dtype='object')

<br>
<br>

Se modifica el nombre de la columna 'Dirección Normalizada'

In [48]:
# Cambio el nombre de la columna 'Dirección Normalizada' a 'LUGAR DEL HECHO'
df_homicidios_hechos = df_homicidios_hechos.rename(columns={'Dirección Normalizada': 'LUGAR DEL HECHO'})

<br>
<br>

La columna 'Cruce' no se elimina para poder estudiar de forma mas fácil si los accidentes ocurren en una intersección de calles o no. Para ello se reemplazan los valores de la columna con SI y NO, según corresponda con el lugar del hecho.

In [49]:
# Se reemplazan los valores NULOS por "No" y los NO NULOS por "Si"
df_homicidios_hechos['Cruce'] = np.where(df_homicidios_hechos['Cruce'].notnull(), 'SI', 'NO')

# Se revisa la columna
df_homicidios_hechos[['Cruce', 'LUGAR DEL HECHO']]

,Cruce,LUGAR DEL HECHO
0,SI,"PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G..."
1,SI,"PAZ, GRAL. AV. y DE LOS CORRALES AV."
2,NO,ENTRE RIOS AV. 2034
3,SI,"LARRAZABAL AV. y VILLEGAS, CONRADO, GRAL."
4,SI,"SAN JUAN AV. y SAENZ PEÃ‘A, LUIS, PRES."
...,...,...
691,SI,RIESTRA AV. y MOM
692,SI,"DELLEPIANE, LUIS, TTE. GRAL. y LACARRA AV."
693,SI,GAONA AV. y TERRADA
694,NO,"PERON, EVA AV. 4071"


<br>
<br>

#### Se estandarizan los nombres de las columnas

In [50]:
# Se coloca la primera en mayúscula
df_homicidios_hechos.columns = [x.capitalize() for x in df_homicidios_hechos.columns]

# Se reemplazan los guiones por espacios
df_homicidios_hechos.columns = df_homicidios_hechos.columns.str.replace('_', ' ')

# Se renombran algunas columnas
df_homicidios_hechos = df_homicidios_hechos.rename(columns={'N victimas': 'Nro Víctimas',
                                                      'Aaaa':'Año',
                                                      'Mm':'Mes',
                                                      'Dd':'Día',
                                                      'Hh':'Hora Estandar',
                                                      'Xy (caba)':'XY (CABA)',
                                                      'Victima':'Víctima',
                                                      'Lugar del hecho':'Lugar del Hecho'})
df_homicidios_hechos.columns

Index(['Id', 'Nro Víctimas', 'Fecha', 'Año', 'Mes', 'Día', 'Hora',
       'Hora Estandar', 'Tipo de calle', 'Cruce', 'Lugar del Hecho', 'Comuna',
       'XY (CABA)', 'Pos x', 'Pos y', 'Participantes', 'Víctima', 'Acusado'],
      dtype='object')

<br>
<br>

#### Se chequean los registros duplicados

In [51]:
filas_duplicadas = df_homicidios_hechos[df_homicidios_hechos.duplicated(subset='Id', keep=False)]
filas_duplicadas

,Id,Nro Víctimas,Fecha,Año,Mes,Día,Hora,Hora Estandar,Tipo de calle,Cruce,Lugar del Hecho,Comuna,XY (CABA),Pos x,Pos y,Participantes,Víctima,Acusado


No se encontraron valores DUPLICADOS.

<br>
<br>

#### Se normalizan los tipos de datos en las columnas 'Hora' y 'Hora Estandar'

In [55]:
# Función que convierte un valor a un objeto de tiempo (time) de Python, si es posible
def convertir_hora(x):
    if isinstance(x, str):
        try:
            return datetime.strptime(x, "%H:%M:%S").time()
        except ValueError:
            return None
    elif isinstance(x, datetime):
        return x.time()
    return x

# Se cambia el tipo de dato en la columna 'Hora'
df_homicidios_hechos['Hora'] = df_homicidios_hechos['Hora'].apply(lambda x: convertir_hora(x))

df_homicidios_hechos[df_homicidios_hechos['Hora'].isna()]

,Id,Nro Víctimas,Fecha,Año,Mes,Día,Hora,Hora Estandar,Tipo de calle,Cruce,Lugar del Hecho,Comuna,XY (CABA),Pos x,Pos y,Participantes,Víctima,Acusado
518,2019-0103,1,2019-12-18,2019,12,18,None,SD,GRAL PAZ,SI,"PAZ, GRAL. AV. y GRIVEO",11,Point (94643.11254058 103831.57115061),-58.52169422,-34.59471640,MOTO-MOTO,MOTO,MOTO


<br>
<br>

Se imputa el dato faltante con la moda

In [56]:
# Se calcula la hora mas común
hora_moda = df_homicidios_hechos['Hora'].mode().iloc[0]

# Se reemplaza el valor None por la hora más común
df_homicidios_hechos['Hora'].fillna(hora_moda, inplace=True)

# Se verifica el tipo de dato para la columna
df_homicidios_hechos['Hora'].apply(type).value_counts()

Hora
<class 'datetime.time'>    696
Name: count, dtype: int64

<br>
<br>

Se imputa tambien en la columna 'Hora Estandar' el dato faltante, con la moda

In [58]:
# Se imputa la hora moda al dato faltante
df_homicidios_hechos['Hora Estandar'] = df_homicidios_hechos['Hora Estandar'].apply(lambda x: int(hora_moda.hour) if x == "SD" else x)

# Se verifica el tipo de dato
df_homicidios_hechos['Hora Estandar'].apply(type).value_counts()

Hora Estandar
<class 'int'>    696
Name: count, dtype: int64

<br>
<br>

#### Se verifican nuevamente los tipos de datos y nulos

In [60]:
dicc = {"Campo": [], "Tipo_de_dato": [], "Nulos": []}

for columna in df_homicidios_hechos.columns:
    dicc["Campo"].append(columna)
    dicc["Tipo_de_dato"].append(df_homicidios_hechos[columna].apply(type).unique())
    dicc["Nulos"].append(df_homicidios_hechos[columna].isnull().sum())

df_info = pd.DataFrame(dicc)
df_info

,Campo,Tipo_de_dato,Nulos
0,Id,[<class 'str'>],0
1,Nro Víctimas,[<class 'int'>],0
2,Fecha,[<class 'pandas._libs.tslibs.timestamps.Timest...,0
3,Año,[<class 'int'>],0
4,Mes,[<class 'int'>],0
5,Día,[<class 'int'>],0
6,Hora,[<class 'datetime.time'>],0
7,Hora Estandar,[<class 'int'>],0
8,Tipo de calle,[<class 'str'>],0
9,Cruce,[<class 'str'>],0


<br>
<br>



<br>
<br>

Se extraen los datos de la pestaña VICTIMAS del excel provisto.

In [53]:
# Se extrae la pestaña VICTIMAS del dataset HOMICIDIOS
df_homicidios_victimas = pd.read_excel(r"C:\Users\Cebol\OneDrive\Escritorio\PI_02_siniestros\Datasets\homicidios.xlsx", sheet_name='VICTIMAS')

# Se imprimen en pantalla las primeras filas
df_homicidios_victimas.head()

,ID_hecho,FECHA,AAAA,MM,DD,ROL,VICTIMA,SEXO,EDAD,FECHA_FALLECIMIENTO
0,2016-0001,2016-01-01,2016,1,1,CONDUCTOR,MOTO,MASCULINO,19,2016-01-01 00:00:00
1,2016-0002,2016-01-02,2016,1,2,CONDUCTOR,AUTO,MASCULINO,70,2016-01-02 00:00:00
2,2016-0003,2016-01-03,2016,1,3,CONDUCTOR,MOTO,MASCULINO,30,2016-01-03 00:00:00
3,2016-0004,2016-01-10,2016,1,10,CONDUCTOR,MOTO,MASCULINO,18,SD
4,2016-0005,2016-01-21,2016,1,21,CONDUCTOR,MOTO,MASCULINO,29,2016-02-01 00:00:00


<br>
<br>

#### Se carga el dataset limpio

In [54]:
df_homicidios.to_csv(r"C:\Users\Cebol\OneDrive\Escritorio\PI_02_siniestros\Datasets\homicidios_limpio.csv", index=False, encoding='utf-8')

NameError: name 'df_homicidios' is not defined